# Fingerprints to SMILES amb MolForge
Transforma una taula amb **fingerprints** en **SMILES** fent servir MolForge.

**Entrada**: CSV amb columna `fingerprints_input_ECPF4`.

**Sortida**: CSV amb columnes `fingerprints_input_ECFP4` i `SMILES_output_ECPF4`.

## Imports

In [1]:
# Per definir l'arrel del projecte
import os

# Pandas pels dataframes
import pandas as pd
import numpy as np # pels NaN

# Per cridar MolForge i guardar-ne l'output
from MolForge import main as molforge_main
import sys
import io
from contextlib import redirect_stdout

## Inputs (part a editar)

Arrel del projecte

In [2]:
os.chdir("/export/home/ddiestre/MolForge_Testing")

Paràmetres de MolForge

In [3]:
FP_NAME = "ECFP4"
MODEL_TYPE = "smiles"  # ["smiles", "selfies"]
DECODE = "greedy"  # ["greedy", "beam"]
CHECKPOINT_NAME = "ECFP4_smiles_checkpoint.pth"

Fitxer de fingerprints preprocessat (path a partir de MolForge_Testing/)

In [4]:
#input_path = "data/MolForge_input/MolForge_MFinput_2000_ECFP4.csv"
#input_path = "data/MolForge_input/CoCoGraph_MFinput_2000_novel.csv"
input_path = "data/MolForge_input/CoCoGraph_MFinput_2000_lt70atoms.csv"

in_col_name = "fingerprints_input_" + FP_NAME

Fitxer en que guardar l'output (path a partir de MolForge_Testing/)

In [5]:
#output_path = "data/MolForge_output/MolForge_MFoutput_2000_ECFP4.csv"
#output_path = "data/MolForge_output/CoCoGraph_MFoutput_2000_novel.csv"
output_path = "data/MolForge_output/CoCoGraph_MFoutput_2000_lt70atoms.csv"

out_col_name = "SMILES_output_" + FP_NAME

## 1. Lectura del fitxer

In [6]:
# Lectura del fitxer
df = pd.read_csv(input_path, sep = ',', index_col = 0)
df.head(5)

,SMILES_input,fingerprints_input_ECFP4
id,,
1,O=C(NCCCCc1nc(-c2ccccn2)cs1)[C@@H]1COCCO1,2 80 193 197 221 282 378 422 448 523 552 592 6...
2,O=C(NCCCCn1cnc([N+](=O)[O-])n1)c1ccc(OC(F)(F)F...,8 80 114 145 197 207 265 317 322 378 486 548 5...
3,O=C(O)C(=O)CC1CCCCC1,2 29 80 223 389 446 484 578 650 715 807 890 92...
4,CC(C)c1ccc([N+](=O)[O-])c(C(C)C)c1N1C(=O)c2c(F...,1 121 146 237 283 314 354 360 437 526 598 633 ...
5,CC(=O)N1CCc2cc(S(=O)(=O)CCC(=O)N3CCN(c4cccc(C)...,80 231 252 319 350 407 432 481 582 626 650 715...


## 2. Execució de Molforge

In [7]:
def run_for_fp(input_fp, fp_name, model_type, checkpoint_name, decode):
    # 0. Tractem els casos "NaN" i "InvalidSMILE"
    if (input_fp == "InvalidSMILE") or (input_fp is np.nan):
        return np.nan

    # 1. Guardar sys.argv original (per restaurar-lo després)
    original_argv = sys.argv
    
    # 2. Crear els arguments com si s'haguessin passat des de CLI (Command Line Interface)
    sys.argv = [
        "",  # el primer argument de sys.argv s'ignora
        f"--input={input_fp}",
        f"--fp={fp_name}",
        f"--model_type={model_type}",
        f"--checkpoint={checkpoint_name}",
        f"--decode={decode}",
    ]
    
    # 3. Capturar stdout
    buf = io.StringIO()  # creem un buffer text en memòria (on guardarem l'output de MolForge)
    try:  # per asegurar-nos que restaurem sys.argv encara que peti
        with redirect_stdout(buf):  # guardem l'output de molforge a buf
            molforge_main()  # executem molforge_main amb els arguments de sys.argv
    finally:
        # 4. Restaurar sys.argv (important en notebooks! tot i que no és estrictament necessari en aquest cas, és una bona pràctica)
        sys.argv = original_argv
    
    # 5. Processar resultat
    output = buf.getvalue()  # transforma buf en string
    for line in output.splitlines():
        line = line.strip()  # elimina espais i salts de línia al principi i al final
        if line.startswith("Result:"):
            return line.split("Result:", 1)[1].strip().replace(" ", "")  # separem per "Result:" i agafem només la part de després fent-li un strip i treient-li espais
    
    return None

In [8]:
smiles_out = [] # aquesta serà la nostra columna "out_col_name"
len_df = len(df)
for i in range(len_df): # de 0 a len_df-1
    s = run_for_fp(df[in_col_name][i+1], FP_NAME, MODEL_TYPE, CHECKPOINT_NAME, DECODE)
    smiles_out.append(s)
    print(f"\r[{i+1}/{len_df}]", end="", flush=True) # Seguiment del progrés

[2000/2000]

## 3. Guardar l'output

In [9]:
# Creem el nou dataframe
df[out_col_name] = smiles_out
df.head(5)

,SMILES_input,fingerprints_input_ECFP4,SMILES_output_ECFP4
id,,,
1,O=C(NCCCCc1nc(-c2ccccn2)cs1)[C@@H]1COCCO1,2 80 193 197 221 282 378 422 448 523 552 592 6...,C1COC(CO1)C(=O)NCCCCC2=NC(=CS2)C3=CC=CC=N3
2,O=C(NCCCCn1cnc([N+](=O)[O-])n1)c1ccc(OC(F)(F)F...,8 80 114 145 197 207 265 317 322 378 486 548 5...,C1=CC(=CC=C1C(=O)NCCCCN2C=NC(=N2)[N+](=O)[O-])...
3,O=C(O)C(=O)CC1CCCCC1,2 29 80 223 389 446 484 578 650 715 807 890 92...,C1CCC(CC1)CC(=O)C(=O)CC(=O)C(=O)CC(=O)C(=O)O
4,CC(C)c1ccc([N+](=O)[O-])c(C(C)C)c1N1C(=O)c2c(F...,1 121 146 237 283 314 354 360 437 526 598 633 ...,CC(C)C1=C(C(=C(C=C1)[N+](=O)[O-])C(C)C)N2C(=O)...
5,CC(=O)N1CCc2cc(S(=O)(=O)CCC(=O)N3CCN(c4cccc(C)...,80 231 252 319 350 407 432 481 582 626 650 715...,CC1=C(C(=CC=C1)N2CCN(CC2)C(=O)CCS(=O)(=O)C3=CC...


In [10]:
df.to_csv(output_path)